In [24]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.formula.api as sm
from sklearn.metrics import mean_squared_error,r2_score,explained_variance_score
from datetime import date
import matplotlib.pyplot as plt
import mysql.connector


def dateFromFloat(date_Float:float):
    day= date_Float % 100
    month = (date_Float // 100)%100 
    year = date_Float // 10000
    return date(year,month,day)


mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="p0o9i8u7",
  database="jockey"
)

features=['draw','addedWeight','declaredHorseWeight','horse_winPer',
                 'horse_winCount','jockey_winPer','jockey_winCount','horse_newDistance','horse_newHorse',
         'horse_last4SpeedRate','horse_latestSpeedRate','Days_from_lastRace']
target=['finishTime']
file0 = "/Users/apple/Workspace/intelliJ/jockey/regression_20190127.csv"
dataset0 = pd.read_csv(file0)

In [34]:
allDates = dataset0.rateDate.unique()
allDates.sort()
allDates

array([20170903, 20170906, 20170910, 20170913, 20170916, 20170920,
       20170924, 20170927, 20171001, 20171005, 20171008, 20171011,
       20171014, 20171018, 20171022, 20171025, 20171029, 20171101,
       20171105, 20171108, 20171111, 20171115, 20171119, 20171122,
       20171126, 20171129, 20171203, 20171206, 20171210, 20171213,
       20171217, 20171220, 20171223, 20171227, 20180101, 20180107,
       20180110, 20180113, 20180117, 20180121, 20180124, 20180128,
       20180131, 20180204, 20180207, 20180210, 20180214, 20180218,
       20180221, 20180225, 20180228, 20180303, 20180307, 20180311,
       20180314, 20180318, 20180321, 20180325, 20180328, 20180402,
       20180408, 20180411, 20180415, 20180418, 20180421, 20180425,
       20180429, 20180502, 20180506, 20180509, 20180512, 20180516,
       20180520, 20180523, 20180527, 20180530, 20180603, 20180606,
       20180610, 20180613, 20180616, 20180624, 20180627, 20180701,
       20180704, 20180708, 20180711, 20180715, 20180902, 20180

In [36]:
allDates = dataset0.rateDate.unique()
allDates.sort()
for raceDate in allDates:
    #if(raceDate <20180218):
    #     continue
    print(raceDate)
    dataset_ofADay = dataset0[(dataset0['rateDate']==raceDate)]
    allSeqs = dataset_ofADay.raceSeqOfDate.unique()
    for seq in allSeqs:
        dataset_test =  dataset_ofADay[(dataset_ofADay['raceSeqOfDate']==seq)]
        distance = dataset_test['distance'].iloc[0]
        raceClass = dataset_test['raceClass'].iloc[0]
        print(distance,raceClass)
        
        dataset_train = dataset0[(dataset0['raceClass']==raceClass) & (dataset0['distance']==distance) 
                        & (dataset0['rateDate']!=raceDate)]
        #dataset_train.shape
        if(dataset_train.size==0):
            continue
        #regression model of lbw or finishTime
        X_train=dataset_train[features]
        y_train=dataset_train[target] #lbw

        X_test=dataset_test[features]
        y_test=dataset_test[target] #lbw


        reg = LinearRegression()
        reg.fit(X_train,y_train)
        y_test_predicted=reg.predict(X_test)
        
        #plt.scatter(y_test, y_test_predicted,  color='black')
        #plt.show()
        #print("Mean squared error: %.2f"
        #      % mean_squared_error(y_test, y_test_predicted))
        # Explained variance score: 1 is perfect prediction
        #print('Variance score: %.2f' % r2_score(y_test, y_test_predicted))

        #explained_variance_score(y_test, y_test_predicted)
        df = pd.DataFrame(data=y_test_predicted, index=y_test.index,columns=['finishTime_predicted'])
        dataset_test_toshow=dataset_test[['rateDate','raceSeqOfDate','distance','raceClass','horseId','place','winOdds','lbw','finishTime']]
        dataset_test_withPredicted=dataset_test_toshow.merge(df,how='inner',left_index=True,right_index=True)
        dataset_test_withPredicted = dataset_test_withPredicted.sort_values(['finishTime_predicted'])
        #dataset_test_withPredicted

        predicted_place=1
        for index, row in dataset_test_withPredicted.iterrows():
            mycursor = mydb.cursor()
            raceDate_str = str(row['rateDate'])
            raceSeqOfDate =str(row['raceSeqOfDate'])
            horseId = str(row['horseId'])
            finishTime_predicted =str(row['finishTime_predicted'])
            ##print("Andrea:",raceDate_str,raceSeqOfDate,horseId,finishTime_predicted)
            sql = ("UPDATE racecard SET predicted_place="+ str(predicted_place) +", predicted_finishTime = "+ finishTime_predicted +
            " WHERE raceDate = "+ raceDate_str + " and raceSeqOfDay = "+raceSeqOfDate + " and horseId ='"+horseId+ "' ")
            
            print(predicted_place)
            predicted_place =predicted_place+1
            #print(sql)
            mycursor.execute(sql)
            mydb.commit()
            print(mycursor.rowcount, " record(s) updated" ,row['raceSeqOfDate'] ,",", row['horseId'] )

20180218
1400 5
1
0  record(s) updated 1 , M273
2
0  record(s) updated 1 , A012
3
0  record(s) updated 1 , V259
4
0  record(s) updated 1 , A059
5
0  record(s) updated 1 , A075
6
0  record(s) updated 1 , V375
7
0  record(s) updated 1 , T329
8
0  record(s) updated 1 , A275
9
0  record(s) updated 1 , S407
10
0  record(s) updated 1 , S113
11
0  record(s) updated 1 , P121
12
0  record(s) updated 1 , T339
13
0  record(s) updated 1 , T383
14
0  record(s) updated 1 , T063
1400 4
1
0  record(s) updated 2 , A363
2
0  record(s) updated 2 , S203
3
0  record(s) updated 2 , B050
4
0  record(s) updated 2 , V250
5
0  record(s) updated 2 , A057
6
0  record(s) updated 2 , A323
7
0  record(s) updated 2 , A273
8
0  record(s) updated 2 , B023
9
0  record(s) updated 2 , S420
10
0  record(s) updated 2 , A311
11
0  record(s) updated 2 , T002
12
0  record(s) updated 2 , B124
13
0  record(s) updated 2 , B076
14
0  record(s) updated 2 , A107
1800 4
1
0  record(s) updated 3 , A278
2
0  record(s) updated 3 , A152


1  record(s) updated 3 , V232
14
1  record(s) updated 3 , S346
1200 4
1
1  record(s) updated 4 , A301
2
1  record(s) updated 4 , B190
3
1  record(s) updated 4 , B119
4
1  record(s) updated 4 , V250
5
1  record(s) updated 4 , T405
6
1  record(s) updated 4 , A179
7
1  record(s) updated 4 , B006
8
1  record(s) updated 4 , B067
9
1  record(s) updated 4 , B069
10
1  record(s) updated 4 , T412
11
1  record(s) updated 4 , B197
12
1  record(s) updated 4 , B201
13
1  record(s) updated 4 , B143
1200 3
1
1  record(s) updated 5 , B040
2
1  record(s) updated 5 , T080
3
1  record(s) updated 5 , A142
4
1  record(s) updated 5 , T200
5
1  record(s) updated 5 , A162
6
1  record(s) updated 5 , A381
7
1  record(s) updated 5 , T392
8
1  record(s) updated 5 , B218
9
1  record(s) updated 5 , V355
10
1  record(s) updated 5 , B168
11
1  record(s) updated 5 , B220
12
1  record(s) updated 5 , B155
13
1  record(s) updated 5 , V093
14
1  record(s) updated 5 , B214
1400 4
1
1  record(s) updated 6 , S098
2
1  record

1  record(s) updated 4 , S324
1600 4
1
1  record(s) updated 5 , T258
2
1  record(s) updated 5 , A045
3
1  record(s) updated 5 , A125
4
1  record(s) updated 5 , A189
5
1  record(s) updated 5 , A081
6
1  record(s) updated 5 , T236
7
1  record(s) updated 5 , A311
8
1  record(s) updated 5 , B038
9
1  record(s) updated 5 , B076
10
1  record(s) updated 5 , A356
11
1  record(s) updated 5 , A392
12
1  record(s) updated 5 , B120
13
1  record(s) updated 5 , A295
1400 4
1
1  record(s) updated 6 , A145
2
1  record(s) updated 6 , A371
3
1  record(s) updated 6 , B124
4
1  record(s) updated 6 , A049
5
1  record(s) updated 6 , V303
6
1  record(s) updated 6 , A348
7
1  record(s) updated 6 , V150
8
1  record(s) updated 6 , V243
9
1  record(s) updated 6 , V234
10
1  record(s) updated 6 , B173
11
1  record(s) updated 6 , V102
12
1  record(s) updated 6 , T288
13
1  record(s) updated 6 , B036
14
1  record(s) updated 6 , A357
1800 2
1
1  record(s) updated 7 , S332
2
1  record(s) updated 7 , B208
3
1  record(

1  record(s) updated 8 , B093
4
1  record(s) updated 8 , A183
5
1  record(s) updated 8 , A030
6
1  record(s) updated 8 , B163
7
1  record(s) updated 8 , B170
8
1  record(s) updated 8 , V355
9
1  record(s) updated 8 , S279
10
1  record(s) updated 8 , V012
11
1  record(s) updated 8 , B096
12
1  record(s) updated 8 , B217
1400 3
1
1  record(s) updated 9 , V266
2
1  record(s) updated 9 , V293
3
1  record(s) updated 9 , V223
4
1  record(s) updated 9 , A245
5
1  record(s) updated 9 , B147
6
1  record(s) updated 9 , A168
7
1  record(s) updated 9 , B071
8
1  record(s) updated 9 , P373
9
1  record(s) updated 9 , A364
10
1  record(s) updated 9 , B181
11
1  record(s) updated 9 , B245
12
1  record(s) updated 9 , V411
13
1  record(s) updated 9 , B223
14
1  record(s) updated 9 , A236
1600 3
1
1  record(s) updated 10 , A127
2
1  record(s) updated 10 , V296
3
1  record(s) updated 10 , V002
4
1  record(s) updated 10 , A065
5
1  record(s) updated 10 , V114
6
1  record(s) updated 10 , A344
7
1  record(s)

1  record(s) updated 1 , A195
11
1  record(s) updated 1 , P324
12
1  record(s) updated 1 , V309
1200 4
1
1  record(s) updated 2 , A270
2
1  record(s) updated 2 , T018
3
1  record(s) updated 2 , A315
4
1  record(s) updated 2 , A209
5
1  record(s) updated 2 , T289
6
1  record(s) updated 2 , S172
7
1  record(s) updated 2 , A347
8
1  record(s) updated 2 , V313
9
1  record(s) updated 2 , A023
10
1  record(s) updated 2 , A281
11
1  record(s) updated 2 , B122
12
1  record(s) updated 2 , B036
1650 4
1
1  record(s) updated 3 , S277
2
1  record(s) updated 3 , V171
3
1  record(s) updated 3 , A049
4
1  record(s) updated 3 , T258
5
1  record(s) updated 3 , S247
6
1  record(s) updated 3 , A239
7
1  record(s) updated 3 , V367
8
1  record(s) updated 3 , S241
9
1  record(s) updated 3 , B124
10
1  record(s) updated 3 , A083
11
1  record(s) updated 3 , T390
12
1  record(s) updated 3 , A233
1200 4
1
1  record(s) updated 4 , T076
2
1  record(s) updated 4 , A340
3
1  record(s) updated 4 , V274
4
1  record(s

...

1  record(s) updated 6 , A204
6
1  record(s) updated 6 , A071
7
1  record(s) updated 6 , T281
8
1  record(s) updated 6 , B113
9
1  record(s) updated 6 , A172
10
1  record(s) updated 6 , A396
11
1  record(s) updated 6 , V336
12
1  record(s) updated 6 , B326
1200 2
1
1  record(s) updated 7 , T146
2
1  record(s) updated 7 , A377
3
1  record(s) updated 7 , A157
4
1  record(s) updated 7 , A248
5
1  record(s) updated 7 , B224
6
1  record(s) updated 7 , V077
7
1  record(s) updated 7 , T235
8
1  record(s) updated 7 , T005
9
1  record(s) updated 7 , V068
10
1  record(s) updated 7 , T186
11
1  record(s) updated 7 , S191
12
1  record(s) updated 7 , A124
1200 3
1
1  record(s) updated 8 , B192
2
1  record(s) updated 8 , A064
3
1  record(s) updated 8 , B147
4
1  record(s) updated 8 , A412
5
1  record(s) updated 8 , V225
6
1  record(s) updated 8 , B015
7
1  record(s) updated 8 , V161
8
1  record(s) updated 8 , T402
9
1  record(s) updated 8 , T213
10
1  record(s) updated 8 , B204
11
1  record(s) updat

1  record(s) updated 2 , V341
4
1  record(s) updated 2 , V088
5
1  record(s) updated 2 , V086
6
1  record(s) updated 2 , S324
7
1  record(s) updated 2 , A281
8
1  record(s) updated 2 , A069
9
1  record(s) updated 2 , A046
10
1  record(s) updated 2 , A369
11
1  record(s) updated 2 , B062
12
1  record(s) updated 2 , B120
1200 4
1
1  record(s) updated 3 , B438
2
1  record(s) updated 3 , B222
3
1  record(s) updated 3 , B463
4
1  record(s) updated 3 , B066
5
1  record(s) updated 3 , C062
6
1  record(s) updated 3 , C015
7
1  record(s) updated 3 , B185
8
1  record(s) updated 3 , C187
9
1  record(s) updated 3 , B379
10
1  record(s) updated 3 , C041
11
1  record(s) updated 3 , C064
12
1  record(s) updated 3 , B324
1400 5
1
1  record(s) updated 4 , B061
2
1  record(s) updated 4 , T140
3
1  record(s) updated 4 , T063
4
1  record(s) updated 4 , T256
5
1  record(s) updated 4 , V319
6
1  record(s) updated 4 , A340
7
1  record(s) updated 4 , A151
8
1  record(s) updated 4 , A399
9
1  record(s) updated

1  record(s) updated 1 , V086
9
1  record(s) updated 1 , B134
10
1  record(s) updated 1 , B041
11
1  record(s) updated 1 , V302
12
1  record(s) updated 1 , A255
1400 5
1
1  record(s) updated 2 , T063
2
1  record(s) updated 2 , V243
3
1  record(s) updated 2 , V202
4
1  record(s) updated 2 , A423
5
1  record(s) updated 2 , T224
6
1  record(s) updated 2 , V367
7
1  record(s) updated 2 , B314
8
1  record(s) updated 2 , V319
9
1  record(s) updated 2 , B121
10
1  record(s) updated 2 , A069
11
1  record(s) updated 2 , P190
12
1  record(s) updated 2 , B130
1000 3
1
1  record(s) updated 3 , B280
2
1  record(s) updated 3 , A257
3
1  record(s) updated 3 , B180
4
1  record(s) updated 3 , V068
5
1  record(s) updated 3 , V077
6
1  record(s) updated 3 , T005
1200 4
1
1  record(s) updated 4 , B364
2
1  record(s) updated 4 , B002
3
1  record(s) updated 4 , B084
4
1  record(s) updated 4 , N360
5
1  record(s) updated 4 , A275
6
1  record(s) updated 4 , A345
7
1  record(s) updated 4 , T381
8
1  record(s) 

1  record(s) updated 5 , B458
12
1  record(s) updated 5 , B439
13
1  record(s) updated 5 , B357
1650 4
1
1  record(s) updated 6 , V102
2
1  record(s) updated 6 , A222
3
1  record(s) updated 6 , B167
4
1  record(s) updated 6 , S131
5
1  record(s) updated 6 , T236
6
1  record(s) updated 6 , V179
7
1  record(s) updated 6 , P421
8
1  record(s) updated 6 , S418
9
1  record(s) updated 6 , A130
10
1  record(s) updated 6 , A325
11
1  record(s) updated 6 , A286
12
1  record(s) updated 6 , V368
13
1  record(s) updated 6 , B353
1800 3
1
1  record(s) updated 7 , T098
2
1  record(s) updated 7 , B129
3
1  record(s) updated 7 , A371
4
1  record(s) updated 7 , B189
5
1  record(s) updated 7 , A416
6
1  record(s) updated 7 , T093
7
1  record(s) updated 7 , T281
8
1  record(s) updated 7 , B336
9
1  record(s) updated 7 , B350
10
1  record(s) updated 7 , A172
11
1  record(s) updated 7 , V184
12
1  record(s) updated 7 , B407
1400 3
1
1  record(s) updated 8 , C013
2
1  record(s) updated 8 , B285
3
1  record(

0  record(s) updated 9 , B094
4
0  record(s) updated 9 , B052
5
0  record(s) updated 9 , V266
6
0  record(s) updated 9 , B421
7
0  record(s) updated 9 , T403
8
0  record(s) updated 9 , A202
9
0  record(s) updated 9 , C095
10
0  record(s) updated 9 , C048
11
0  record(s) updated 9 , B270
12
0  record(s) updated 9 , B302
13
0  record(s) updated 9 , C121
14
0  record(s) updated 9 , B409
1400 2
1
0  record(s) updated 10 , B088
2
0  record(s) updated 10 , B244
3
0  record(s) updated 10 , T400
4
0  record(s) updated 10 , V040
5
0  record(s) updated 10 , A050
6
0  record(s) updated 10 , S253
7
0  record(s) updated 10 , B224
8
0  record(s) updated 10 , C150
9
0  record(s) updated 10 , C164
10
0  record(s) updated 10 , A030
11
0  record(s) updated 10 , C145
12
0  record(s) updated 10 , C001
13
0  record(s) updated 10 , V142
14
0  record(s) updated 10 , A188
20190123
2200 5
1
1  record(s) updated 1 , A285
2
1  record(s) updated 1 , A249
3
1  record(s) updated 1 , S205
4
1  record(s) updated 1 , 